In [6]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# Create a Flask application instance
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///predictions.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Create a SQLAlchemy instance
db = SQLAlchemy(app)

# Define the Prediction model
class Prediction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_name = db.Column(db.String(50), nullable=False)
    vote_data = db.Column(db.PickleType, nullable=False)

    def __repr__(self):
        return f'<Prediction {self.user_name}>'

# Function to retrieve all predictions from the database
def get_all_predictions():
    return Prediction.query.all()

# Function to delete a prediction by user name
def delete_prediction_by_user_name(user_name):
    prediction = Prediction.query.filter_by(user_name=user_name).first()
    if prediction:
        db.session.delete(prediction)
        db.session.commit()
        return f"Prediction for user {user_name} deleted successfully."
    else:
        return f"No prediction found for user {user_name}."

# Function to delete all predictions from the database
def delete_all_predictions():
    num_deleted = Prediction.query.delete()
    db.session.commit()
    return f"Deleted {num_deleted} predictions."

# Connect to the database
with app.app_context():
    # Example: Retrieve all predictions
    predictions = get_all_predictions()
    print("All Predictions:")
    for prediction in predictions:
        print(prediction.user_name, prediction.vote_data)

    # Example: Delete a prediction by user name
    user_to_delete = "test"  # Replace with the actual user name
    result = delete_prediction_by_user_name(user_to_delete)
    print(result)


All Predictions:
Leo {'Best Picture': 'American Fiction – Ben LeClair, Nikos Karamigios, Cord Jefferson and Jermaine Johnson, producers', 'Best Director': 'Martin Scorsese – Killers of the Flower Moon'}
jean {'Best Picture': 'Barbie – David Heyman, Margot Robbie, Tom Ackerley and Robbie Brenner, producers', 'Best Director': 'Justine Triet – Anatomy of a Fall'}
No prediction found for user test.


In [1]:
from flask import Flask, render_template, request,  redirect, url_for
from bs4 import BeautifulSoup
import requests
import pandas as pd
from warnings import filterwarnings 
filterwarnings("ignore")

from flask_sqlalchemy import SQLAlchemy


app = Flask(__name__, static_url_path='/static')
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///predictions.db'  # SQLite database file
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)


# Dictionary to store user votes
user_votes = {}


def scrap_nominees(url="https://en.wikipedia.org/wiki/96th_Academy_Awards", keep='all'):
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    indiatable=soup.find('table',{'class':"wikitable"}, role="presentation")
    
    # Create an empty DataFrame to store the nominees
    df_nominees = pd.DataFrame(columns=['category', 'nominees'])

    # Extract rows from the table
    rows = indiatable.find_all('tr')

    # Iterate through rows and print the content
    for row in rows:
        # Extract cells from the row
        cells = row.find_all(['th', 'td'])
        
        # Extract text from each cell
        cell_data = [cell.get_text(strip=False) for cell in cells]
        
        for col in [0,1]:
            try:
                split_cell = cell_data[col].split('\n')[:-1]
            except:
                continue
            category = split_cell[0]
            nominees = split_cell[1:]
            
            df_nominees = df_nominees.append({'category': category, 'nominees': nominees}, ignore_index=True)

    nominees_dict = dict(zip(df_nominees['category'], df_nominees['nominees']))
    

In [3]:
url="https://en.wikipedia.org/wiki/96th_Academy_Awards"
keep='all'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

indiatable=soup.find('table',{'class':"wikitable"}, role="presentation")

# Create an empty DataFrame to store the nominees
df_nominees = pd.DataFrame(columns=['category', 'nominees'])

# Extract rows from the table
rows = indiatable.find_all('tr')

# Iterate through rows and print the content
for row in rows:
    # Extract cells from the row
    cells = row.find_all(['th', 'td'])
    
    # Extract text from each cell
    cell_data = [cell.get_text(strip=False) for cell in cells]
    
    for col in [0,1]:
        try:
            split_cell = cell_data[col].split('\n')[:-1]
        except:
            continue
        category = split_cell[0]
        nominees = split_cell[1:]
        
        df_nominees = df_nominees.append({'category': category, 'nominees': nominees}, ignore_index=True)

NameError: name 'url' is not defined